In [ ]:
pip install bitsandbytes transformers accelerate peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 5.2 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 3.4 MB/s eta 0:00:005.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 2.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch 
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(
      "facebook/opt-350m", 
      quantization_config=quantization_config, 
      torch_dtype=torch.float32
)

2025-03-02 16:01:25.123855: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-02 16:01:25.231959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740911485.277550    6444 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740911485.289051    6444 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 16:01:25.381783: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import torch
from bitsandbytes.functional import quantize_nf4

x = torch.randn(4, 4).cuda()
x_qnf4, state = quantize_nf4(x)
print(x_qnf4)


tensor([[239],
        [226],
        [115],
        [229],
        [203],
        [234],
        [140],
        [240]], device='cuda:0', dtype=torch.uint8)


In [3]:
!pip install torch torchvision torchaudio
!pip install transformers datasets accelerate peft bitsandbytes wandb trl safetensors
!pip install -U huggingface_hub


In [4]:
from huggingface_hub import login
login()  # Enter your token when prompted


In [8]:
import pandas as pd
from datasets import load_dataset

# Option 1: Load Alpaca dataset
dataset = load_dataset("tatsu-lab/alpaca")

# Option 2: Load from a custom CSV/JSON file
# dataset = load_dataset('json', data_files='your_data.json')

# View dataset structure
print(dataset["train"][0])

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}


In [11]:
def format_prompt(example):
    """Format the prompt for Llama-3 models"""
    if "instruction" in example and "input" in example and "output" in example:
        # Format: <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        # You are a helpful assistant.<|eot_id|>
        # <|start_header_id|>user<|end_header_id|>
        # {instruction} {input}<|eot_id|>
        # <|start_header_id|>assistant<|end_header_id|>
        # {output}<|eot_id|>
        
        system_prompt = "You are a helpful assistant."
        instruction = example["instruction"]
        input_text = example["input"] if example["input"] else ""
        output = example["output"]
        
        if input_text:
            formatted_prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{instruction}\n{input_text}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n{output}<|eot_id|>"
        else:
            formatted_prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{instruction}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n{output}<|eot_id|>"
            
        example["text"] = formatted_prompt
        return example
    else:
        # Handle other dataset formats
        raise ValueError("Dataset format not supported. Need 'instruction', 'input', and 'output' fields.")

# Apply formatting to the dataset
formatted_dataset = dataset.map(format_prompt)

# Split into training and validation sets
split_dataset = formatted_dataset["train"].train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Print GPU memory before loading the model
print(f"GPU memory before loading model: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model in 4-bit precision
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Print GPU memory after loading the model
print(f"GPU memory after loading model: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

GPU memory before loading model: 0.39 GB


model-00002-of-00004.safetensors:  53%|#####2    | 2.63G/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ba/83/ba837357c37c4c572f89ebcdffc2867477dbb4768779d332eb303f848c60d0de/d9eee5f23d94405d90b7e9ff88b9443fee42f8528a658f54214c2aba7530d80c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1740915143&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDkxNTE0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2JhLzgzL2JhODM3MzU3YzM3YzRjNTcyZjg5ZWJjZGZmYzI4Njc0NzdkYmI0NzY4Nzc5ZDMzMmViMzAzZjg0OGM2MGQwZGUvZDllZWU1ZjIzZDk0NDA1ZDkwYjdlOWZmODhiOTQ0M2ZlZTQyZjg1MjhhNjU4ZjU0MjE0YzJhYmE3NTMwZDgwYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=H30cemMvRD48QbDh9XtJnlCO6NCKtKv600ZMR7pyDnINNF7RX3d1JKps07ww4gbtRUX3vLcjPhUpELmGr2jGLVNiWZ8%7ElVey6gU3dWvL0LNhm61mlUuqzlg%7EQvdOaWTfvFOvfaVLDiJC1KxomjxP8Mqi-f7F5nSDGZzV-zIvAlJuYJp3hYL6owIEvUtVwggK6Y-Nc8Qa0Iis7KwzMgl8yz%7EJwhudt-43

model-00002-of-00004.safetensors:  65%|######5   | 3.25G/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ba/83/ba837357c37c4c572f89ebcdffc2867477dbb4768779d332eb303f848c60d0de/d9eee5f23d94405d90b7e9ff88b9443fee42f8528a658f54214c2aba7530d80c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1740915143&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDkxNTE0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2JhLzgzL2JhODM3MzU3YzM3YzRjNTcyZjg5ZWJjZGZmYzI4Njc0NzdkYmI0NzY4Nzc5ZDMzMmViMzAzZjg0OGM2MGQwZGUvZDllZWU1ZjIzZDk0NDA1ZDkwYjdlOWZmODhiOTQ0M2ZlZTQyZjg1MjhhNjU4ZjU0MjE0YzJhYmE3NTMwZDgwYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=H30cemMvRD48QbDh9XtJnlCO6NCKtKv600ZMR7pyDnINNF7RX3d1JKps07ww4gbtRUX3vLcjPhUpELmGr2jGLVNiWZ8%7ElVey6gU3dWvL0LNhm61mlUuqzlg%7EQvdOaWTfvFOvfaVLDiJC1KxomjxP8Mqi-f7F5nSDGZzV-zIvAlJuYJp3hYL6owIEvUtVwggK6Y-Nc8Qa0Iis7KwzMgl8yz%7EJwhudt-43

model-00002-of-00004.safetensors:  70%|######9   | 3.49G/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ba/83/ba837357c37c4c572f89ebcdffc2867477dbb4768779d332eb303f848c60d0de/d9eee5f23d94405d90b7e9ff88b9443fee42f8528a658f54214c2aba7530d80c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1740915143&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDkxNTE0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2JhLzgzL2JhODM3MzU3YzM3YzRjNTcyZjg5ZWJjZGZmYzI4Njc0NzdkYmI0NzY4Nzc5ZDMzMmViMzAzZjg0OGM2MGQwZGUvZDllZWU1ZjIzZDk0NDA1ZDkwYjdlOWZmODhiOTQ0M2ZlZTQyZjg1MjhhNjU4ZjU0MjE0YzJhYmE3NTMwZDgwYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=H30cemMvRD48QbDh9XtJnlCO6NCKtKv600ZMR7pyDnINNF7RX3d1JKps07ww4gbtRUX3vLcjPhUpELmGr2jGLVNiWZ8%7ElVey6gU3dWvL0LNhm61mlUuqzlg%7EQvdOaWTfvFOvfaVLDiJC1KxomjxP8Mqi-f7F5nSDGZzV-zIvAlJuYJp3hYL6owIEvUtVwggK6Y-Nc8Qa0Iis7KwzMgl8yz%7EJwhudt-43

model-00002-of-00004.safetensors:  70%|#######   | 3.52G/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ba/83/ba837357c37c4c572f89ebcdffc2867477dbb4768779d332eb303f848c60d0de/d9eee5f23d94405d90b7e9ff88b9443fee42f8528a658f54214c2aba7530d80c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1740915143&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDkxNTE0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2JhLzgzL2JhODM3MzU3YzM3YzRjNTcyZjg5ZWJjZGZmYzI4Njc0NzdkYmI0NzY4Nzc5ZDMzMmViMzAzZjg0OGM2MGQwZGUvZDllZWU1ZjIzZDk0NDA1ZDkwYjdlOWZmODhiOTQ0M2ZlZTQyZjg1MjhhNjU4ZjU0MjE0YzJhYmE3NTMwZDgwYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=H30cemMvRD48QbDh9XtJnlCO6NCKtKv600ZMR7pyDnINNF7RX3d1JKps07ww4gbtRUX3vLcjPhUpELmGr2jGLVNiWZ8%7ElVey6gU3dWvL0LNhm61mlUuqzlg%7EQvdOaWTfvFOvfaVLDiJC1KxomjxP8Mqi-f7F5nSDGZzV-zIvAlJuYJp3hYL6owIEvUtVwggK6Y-Nc8Qa0Iis7KwzMgl8yz%7EJwhudt-43

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs-us-1.hf.co\', port=443): Max retries exceeded with url: /repos/ba/83/ba837357c37c4c572f89ebcdffc2867477dbb4768779d332eb303f848c60d0de/d9eee5f23d94405d90b7e9ff88b9443fee42f8528a658f54214c2aba7530d80c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1740915143&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDkxNTE0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2JhLzgzL2JhODM3MzU3YzM3YzRjNTcyZjg5ZWJjZGZmYzI4Njc0NzdkYmI0NzY4Nzc5ZDMzMmViMzAzZjg0OGM2MGQwZGUvZDllZWU1ZjIzZDk0NDA1ZDkwYjdlOWZmODhiOTQ0M2ZlZTQyZjg1MjhhNjU4ZjU0MjE0YzJhYmE3NTMwZDgwYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=H30cemMvRD48QbDh9XtJnlCO6NCKtKv600ZMR7pyDnINNF7RX3d1JKps07ww4gbtRUX3vLcjPhUpELmGr2jGLVNiWZ8~lVey6gU3dWvL0LNhm61mlUuqzlg~QvdOaWTfvFOvfaVLDiJC1KxomjxP8Mqi-f7F5nSDG